# Übung 07: Gruppieren
Im folgenden Abschnitt wollen wir uns das Olympiadatenset noch genauer anschauen. Hierfür benötigen wir die Datei 
* <font color=orange>bigdata/120-years-of-olympic-history-athletes-and-results/athlete_events.csv</font>

**<font color='red'> Achtung:</font>** bis 1980 wurden die Winterspiele alle zwei Jahre ausgetragen. Dh. in manchen Jahren gibt es sowohl Winter als auch Sommerspiele!</font>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('../../src/bigdata/120-years-of-olympic-history-athletes-and-results/athlete_events.csv')
df.head()

In [ ]:
# Übersicht über den DataFrame erhalten
df.info()

## Teilaufgabe 1:
Zunächst wollen wir erstmal ein paar grundliegende Fragen beantworten zum Datenset beantworten. Dabei hilft uns die Funktion ``.groupby``. Es stellen sich folgende Fragen:

* Wie viele Frauen und Männer insgesamt schon an den Olympischen Spielen teilgenommen haben
* Durchschnittsalter der Männer und Frauen
* Maximalalter der Männer und der Frauen
* Wie viele Teilnehmer haben pro Jahr an den Olympischen Sommer Spielen teilgenommen 
* Aus welchen 5 Ländern kammen die meisten Teilnehmer bei den Sommerspielen 2008
* Wie viele Frauen und Männer in jedem Jahr an den Olympischen Spielen teilgenommen (Unterscheide zusätzlich zwischen Winter und Sommer)

*Hinweis zur Vereinfachung: Jede Teilnahme an einem Wettkampf zählt, das heißt, ein Athlet kann mehrfach gezählt werden*

**Wie viele Frauen und Männer insgesamt schon an den Olympischen Spielen teilgenommen haben**

In [ ]:
df.groupby('Sex').size()

**Durchschnittsalter der Männer und Frauen**

In [ ]:
df[['Sex', 'Age']].groupby('Sex').mean()

**Maximalalter der Männer und der Frauen**

In [ ]:
df[['Sex', 'Age']].groupby('Sex').max()

**Wie viele Frauen und Männer in jedem Jahr an den Olympischen Spielen teilgenommen haben**

Hinweis: Mit ``.reset_index()`` können wir den GroupBy in einen DataFrame zurückwandeln

In [ ]:
df.groupby(['Year', 'Season', 'Sex']).size().reset_index().head()

**Wie viele Teilnehmer haben pro Jahr an den Olympischen Sommer Spielen teilgenommen**

In [ ]:
df_summer = df[df['Season'] == 'Summer']
df_summer = df_summer.groupby(['Year']).size()
df_summer = df_summer.reset_index()
df_summer = df_summer.rename(columns={0:'Count'})
df_summer.tail()

**Aus welchen 5 Ländern kammen die meisten Teilnehmer bei den Sommerspielen 2008** 

In [ ]:
df_summer2 = df[(df['Season'] == 'Summer') & (df['Year'] == 2008)]
df_summer2 = df_summer2.groupby(['Year', 'Team']).size().reset_index()
df_summer2 = df_summer2.rename(columns={0:'Count'})
df_summer2.sort_values('Count', ascending=False).head()

## Teilaufgabe 2:
Wir wollen herausfinden welcher Athlet der/die erfolgreichste war. Erstelle dafür einen neune Dataframe mit dem folgenden Format:

| I |ID|Name|Medal|
|---|---|---|---|
|1|4|Edgar Lindenau Aabye|1|
|2|17|Paavo Johannes Aaltonen|3|
|3|20|Kjetil Andr Aamodt|4|
|4|21|Ragnhild Margrethe Aamodt|1|
|5|40|Roald Edgar Aas|1|

**Hinweis**: *Das I in der Tabelle oben igonieren. Die Überschrift des Index ist wie gewohnt leer.(Geht leider nicht anderes darzustellen)*

Sortiere die Ergebnisse anschließend so, das die Person mit der höchsten Anzahl an goldenen Medalien ganz oben steht.




In [ ]:
df_gold = df[df['Medal'] == 'Gold']
df_gold = df_gold[['ID', 'Medal', 'Name']].groupby(['ID', 'Name']).count()
df_gold = df_gold.reset_index()
df_gold.sort_values(['Medal'], ascending=False).head()

### Teilaufgabe 3:
Wir interessieren uns nun für den Medallienspiegel der Atleten. Fasse diesen mit Groupby zusammen und lass dir sortiert nach der Anzahl der Gold, Silber und Bronzemedallien die Athleten ausgeben.

Bereinige außerdem die **NaN** Werte in der Spalte Medal indem du sie durch ein passenden Begriff ersetzt und auch diese Werte in der Auswertung vorkommen.

**Tipp**: Du kannst anstatt ``.reset_index()`` auch ``.unstack()`` benutzen. Schaue dir an welchen Effekt das auf deinen DataFrame hat.

In [ ]:
# Kopieren des Dataframes, um Warnmeldung zu vermeiden
df_medals = df.copy()

# Bereinigen
df_medals['Medal'] = df_medals['Medal'].fillna('Teilnahme')
df_medals = df_medals.groupby(['ID', 'Name' , 'Medal']).size().unstack()

# Optionale Schönheitskorrektur
df_medals = df_medals[['Gold', 'Silver', 'Bronze', 'Teilnahme']]
df_medals = df_medals.fillna(0)

# Auswertung sortieren
df_medals.sort_values(['Gold', 'Bronze', 'Silver'], ascending=False).head()

### Teilaufgabe 4:
Beim Gruppieren bleibt die urspüngliche Sortierung des DataFrames erhalten. Für manche Auswertungen ist es daher notwendig, das der Dataframe vorher korrekt sortiert wurde.

Finde für jeden Athleten raus, welches sein letztes Jahr war, in dem er oder sie angetreten ist. Überprüfe die Lösung indem du zwei bekannte Olympioniken im Internet nachschlägst:

* Michael Fred Phelps, II
* Raymond Clarence "Ray" Ewry

In [ ]:
df_last = df.sort_values(['ID', 'Year']).groupby('ID').last()
df_last.head()

In [ ]:
df_last[df_last['Name'] == 'Michael Fred Phelps, II']

In [ ]:
df_last[df_last['Name'] == 'Raymond Clarence "Ray" Ewry']

## Zusatzaufgabe 1:
Finde heraus wer die meisten Medallien (unabhänigig ob Gold, Silber oder Bronze) in der jeweiligen Sportkategorie (Spalte Sport) gewonnen hat.

In [ ]:
# .copy() ist neu. Wir machen das hier nur um eine Warnmeldung zu unterdrücken. 
# Was es im Detail damit auf sich hat, kannst du den Link in der Warnmeldung entnehmen.
df_sport = df.copy()
df_sport = df_sport.dropna(subset=['Medal'])
df_sport = df_sport.groupby(['Sport' , 'ID', 'Name']).size().reset_index()
df_sport = df_sport.rename(columns={0:'Count'})
df_sport = df_sport.sort_values('Count', ascending=False)
df_sport.groupby('Sport').first().head()

## Zusatzaufgabe 2:
Ermittle jeweils den zweitbesten in jeder Sportkategorie. Außerdem finde den den zweitältersten in jeder Kategorie heraus.

**Tipp**: Vielleicht wirst du in der Übersicht der Funktionen für ein GroupBy fündig. <br>
https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats

**Zweitbeste Person**

In [ ]:
# An dieser Stelle können wir den DataFrame aus Zusatzaufgabe 1 wiederverwenden
df_sport.groupby('Sport').nth(2).head()

**Zweitälterste Person**

In [ ]:
# An dieser Stelle können wir den DataFrame aus Zusatzaufgabe 1 wiederverwenden
df_oldest = df[['ID', 'Name', 'Age', 'Sport']].sort_values(['ID', 'Age'])
df_oldest = df_oldest.dropna(subset=['Age'])
df_oldest = df_oldest.groupby(['ID', 'Name', 'Sport']).last().reset_index()
df_oldest = df_oldest.sort_values(['Sport', 'Age'])
df_oldest.groupby('Sport').nth(-2).head()

# Zusatzaufgabe 3
Nutze ``.drop_duplicates()``, so dass jeder Athlet in der Teilaufgabe 1 nur ein einziges Mal in die Statistik einfließt.

In [ ]:
# Nutze die beiden folgenden bereinigten DataFrames je nach Situation
df = df.drop_duplicates(subset=['ID', 'Year' ,'Season'])
df = df.drop_duplicates(subset=['ID'])